## Shor's Algorithm

In this notebook, Shor's algorithm is implemented for an n digit number using m qubits. The implementation has several stages:

- Classical attempt to factor N
- Quantum Order-Finding Subroutine
    - Quantum Phase Estimation
    - Continued Fractions Algorithm
 
We will start with the classical attempt to factor N, which consists of Euclid's algorithm and an evenness check. We select some random integer a for $2 ≤ a < N$. If a and N share a non-trivial (not equal to 1) factor, then we have factored N.

In [1]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
from qiskit import BasicAer, IBMQ
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import plot_histogram
import typing

In [9]:
# classical functions

def lin_expr(high, low):
    a = high // low
    b = high - (a * low)
    #print(str(high) + " = " + str(a) + " * " + str(low) + " + " + str(b))
    return low, b 

def euclid(integers: typing.List[int]) -> int:
    """
    takes any size list of integers, returns the greatest common factor of
    all integers.

    parameters:
    - integers: a list of positive integers
    """
    num_ints = len(integers)
    sorted_ints = [abs(x) for x in integers]
    sorted_ints = sorted(sorted_ints) # sorted from least to greatest
    ls1 = 0 # where we draw from the equation intY = A*intX + B
    rs1 = 0 #                                ls1  = rs1    + rs2
    rs2 = 0
    
    if (num_ints <= 1):
        print("Invalid input for gcd function.")
        return
    else:
        first_int = sorted_ints.pop(0)
        for int in sorted_ints:
            rs1 = int
            rs2 = first_int
            while (rs2 != 0):
                rs1, rs2 = lin_expr(rs1, rs2)
            first_int = rs1
        return first_int

def get_random_int(N):
    """
    takes a large two prime product N, returns a random integer
    a where 2 ≤ a < N

    parameters:
    - N: a large product of two prime numbers
    """
    random_integer = np.random.randint(low=2, high=N)
    return random_integer

In [12]:
# numbers which are factors of n
p = 3
q = 5

# multiply together to get N
N = p * q

# begin classical stage 1
a = get_random_int(N)
print(f'a = {a}')

gcd = euclid([a, N])
print(f'gcd({a},{N}) = {gcd}')


a = 13
gcd(13,15) = 1
